In [1]:
import pandas as pd
import numpy as np

In [2]:
#df_customer_lifespan_data= pd.read_excel('https://docs.google.com/spreadsheets/d/1by8tPHwOnq3uKYK2E7sA9VBUYoPM4p1Rnrm_Ss9cyHI/export?format=xlsx')
#df_daily_marketing_spendings= pd.read_excel('https://docs.google.com/spreadsheets/d/1AZOIThOV4P-0eYDge53ZwumVkfkHoYPWxst3k3Bv87c/export?format=xlsx')
#df_monthly_expenses= pd.read_excel('https://docs.google.com/spreadsheets/d/10OGbaywwMIqKgnPGy8VDvpBVtjyqln47iYa2lFhI9Mw/export?format=xlsx')
#df_payroll= pd.read_excel('https://docs.google.com/spreadsheets/d/1c_WihqTZCQvNgxzmd-OwhR9i5diwtfxXVLyMn8R-Lp4/export?format=xlsx')
#df_receipts_history= pd.read_excel('https://docs.google.com/spreadsheets/d/1qayqML1zCKdmtzutkcy9LWvE6xFRm6TGBEVkHHJKIuE/export?format=xlsx')


# **Extracting data on March, 2023**

In [3]:
def read_file_ggsheet(file_id,month_col = None):
  df_ggsheet= pd.read_excel("https://docs.google.com/spreadsheets/d/"+ file_id +"/export?format=xlsx")
  if month_col is not None:
    df_202303 = df_ggsheet[(df_ggsheet[month_col].dt.month == 3) & (df_ggsheet[month_col].dt.year == 2023)]
    return df_202303
  else:
    return df_ggsheet


In [4]:
#ko áp dụng dc cho customer life span
df_customer_lifespan_data= read_file_ggsheet('1by8tPHwOnq3uKYK2E7sA9VBUYoPM4p1Rnrm_Ss9cyHI')
df_daily_marketing_spendings= read_file_ggsheet('1AZOIThOV4P-0eYDge53ZwumVkfkHoYPWxst3k3Bv87c', month_col = "date")
df_monthly_expenses= read_file_ggsheet('10OGbaywwMIqKgnPGy8VDvpBVtjyqln47iYa2lFhI9Mw', month_col = "month")
df_payroll= read_file_ggsheet('1c_WihqTZCQvNgxzmd-OwhR9i5diwtfxXVLyMn8R-Lp4', month_col = "month")
df_receipts_history= read_file_ggsheet('1qayqML1zCKdmtzutkcy9LWvE6xFRm6TGBEVkHHJKIuE', month_col = "date")


In [5]:
df_receipts_history.head(5)

,date,customer_id,receipt_amount,new_customer
618,2023-03-01,1062,103,0
619,2023-03-01,2243,157,0
620,2023-03-01,1166,372,0
621,2023-03-01,2406,426,1
622,2023-03-01,2761,41,1


# **1. Customer Acquisition Cost (CAC) – "How much do I spend to get a customer?"**

## Total Sale and Mkt expenses

- Online advertising: daily_marketing_spendings -> sum spending
- Sales staff salaries and commissions: payroll -> department in sales, marketing
- Marketing software: monthly expenses -> item = salesforce
- Content creation: nó thuộc phần online advertising rùi và payroll

In [6]:
online_ads_cost = df_daily_marketing_spendings['spending'].sum()
sale_salary = df_payroll[df_payroll['department'].isin(['sales','marketing'])]['paid'].sum()
mkt_software_cost = df_monthly_expenses[df_monthly_expenses['item'] == 'salesforce']['amount'].sum()
total_sale_mkt_expenses = online_ads_cost + sale_salary + mkt_software_cost

## Number of new customers

In [7]:
new_customer = df_receipts_history[df_receipts_history['new_customer'] == 1].nunique()['customer_id']

In [8]:
new_customer

63

## CAC

In [9]:
CAC = total_sale_mkt_expenses / new_customer
CAC

1092.5396825396826

## **2. Average Revenue Per User (ARPU) – "How much money do I make per customer?"**

- Total Rev: receipts_history -> receipt_amount sum
- num cust: receipts_history -> customer_id count unique

In [10]:
#total rev / #user
total_rev = df_receipts_history['receipt_amount'].sum()
num_cust = df_receipts_history['customer_id'].nunique()
ARPU = total_rev / num_cust
ARPU

284.3595890410959

## **3.  Cost of Goods Sold (COGS)– "How much does it cost me to produce my product?"**



*   Server cost + Software licenses: monthly expenses -> category
*   Salary of the employees that directly involved into product development: payroll -> department = Engineering.

In [11]:
#COGS
infrastructure_costs = df_monthly_expenses[df_monthly_expenses['category'].isin(['Server Costs','Software Licenses'])]['amount'].sum()
salary_employees_involved = df_payroll[df_payroll['department'] == 'Engineering']['paid'].sum()

In [12]:
COGS = infrastructure_costs + salary_employees_involved
COGS

22540

##**4. Gross Margin – "How much do I actually keep after covering production costs?"**

Gross Margin (%) = (Revenue - COGS)/Revenue x 100

In [13]:
gross_margin = (total_rev - COGS) / total_rev * 100
gross_margin

72.8541664157624

##**5. Lifetime Value (LTV) – "How much money will a customer bring me over time?"**

LTV = ARPU * CustomerLifeSpan * Gross Margin

In [14]:
df_customer_lifespan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  100 non-null    int64         
 1   start_date  100 non-null    datetime64[ns]
 2   churn_date  100 non-null    datetime64[ns]
dtypes: datetime64[ns](2), int64(1)
memory usage: 2.5 KB


In [15]:
df_customer_lifespan_data['life_span(months)'] = (((df_customer_lifespan_data['churn_date'] - df_customer_lifespan_data['start_date']).dt.days )/30).astype('int')


In [20]:
df_customer_lifespan_data.head()

,Unnamed: 0,start_date,churn_date,life_span(months)
0,1000,2021-11-15,2022-09-14,10
1,1001,2022-04-15,2023-02-16,10
2,1002,2022-10-30,2023-02-04,3
3,1003,2021-08-22,2023-02-07,17
4,1004,2021-08-23,2022-02-02,5


In [23]:
LTV = ARPU*df_customer_lifespan_data['life_span(months)'].mean()*(gross_margin/100)
LTV

1930.8039726027398

##**6. LTV/CAC Ratio – "Is my business profitable in the long run?"**

In [24]:
ltv_cac = LTV / CAC
ltv_cac

1.767262098997132